## 실습 [11-1]  

### 1. 실습명 : N-gram 언어 모델로 문장 생성하기
### 2. 실습 목적 및 설명
 
*   파이썬의 NLTK 패키지를 이용하여 N-gram 언어 모델을 구축한다
*   네이버에서 오픈 소스로 제공하는 nsmc 영화 리뷰 데이터셋을 이용해 문장을 생성한다.

### 3. 참조: 자연어처리바이블 9장

##### 0) 라이브러리 불러오기

In [8]:
""" 이건 Ubuntu나 Colab 환경인듯
# 한국어 처리에 필요한 konlpy 패키지를 설치하기 전에 선행 파일을 설치한다. 
!apt-get update

!apt-get install g++ openjdk-8-jdk python-dev python3-dev

!pip3 install JPype1-py3

!pip3 install konlpy

!JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"
"""

'apt-get'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'apt-get'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


'JAVA_HOME'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [24]:
# (1) 기본 라이브러리
import random
import numpy as np
import codecs
from tqdm import tqdm
import urllib.request # !!

# (2) 자연어처리 라이브러리
import nltk
from nltk.util import ngrams
from nltk import word_tokenize
from nltk import ConditionalFreqDist
from nltk.probability import ConditionalProbDist, MLEProbDist

from konlpy.tag import Okt

nltk.download('punkt') # NLTK 사용을 위한 선행 패키지 설치

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

##### 1) 데이터 불러오기

In [25]:
# (1) 데이터 불러오기
sentence = "나는 매일 아침 지하철을 탄다"

##### 2) Tokenization

In [26]:
# (1) Tokenization : 띄어쓰기 기준 토큰화
sentence_nltk = word_tokenize(sentence)

print(f'sentence_nltk : {sentence_nltk}')

sentence_nltk : ['나는', '매일', '아침', '지하철을', '탄다']


In [27]:
# (1) Tokenization : 형태소 기준 토큰화 (한국어는 띄어쓰기 기준으로 단어를 쪼개지 않기 때문)
tagger = Okt()

def okt_tokenize(sentence):
  sentence_okt = ['/'.join(t) for t in tagger.pos(sentence)]
  # print(tagger.pos(sentence))
  return sentence_okt

sentence_okt = okt_tokenize(sentence)

print(f'sentence_okt : {sentence_okt}')

sentence_okt : ['나/Noun', '는/Josa', '매일/Noun', '아침/Noun', '지하철/Noun', '을/Josa', '탄다/Verb']


##### 3) N-gram

(1) N-gram 변환

In [28]:
bigram = ngrams(sentence_okt, 2)
trigram = ngrams(sentence_okt, 3)

In [29]:
print("bigram: ")
for b in bigram:
  print(b)

print("\ntrigram: ")
for t in trigram:
  print(t)

bigram: 
('나/Noun', '는/Josa')
('는/Josa', '매일/Noun')
('매일/Noun', '아침/Noun')
('아침/Noun', '지하철/Noun')
('지하철/Noun', '을/Josa')
('을/Josa', '탄다/Verb')

trigram: 
('나/Noun', '는/Josa', '매일/Noun')
('는/Josa', '매일/Noun', '아침/Noun')
('매일/Noun', '아침/Noun', '지하철/Noun')
('아침/Noun', '지하철/Noun', '을/Josa')
('지하철/Noun', '을/Josa', '탄다/Verb')


(2) Padding 추가 : 입력 데이터 시작, 끝을 알리는 토큰 추가

In [30]:
bigram = ngrams(sentence_okt, 2, pad_left=True, pad_right=True, left_pad_symbol="<s>", right_pad_symbol="</s>")

In [31]:
print("bigrams with padding: ")
for b in bigram:
  print(b)

bigrams with padding: 
('<s>', '나/Noun')
('나/Noun', '는/Josa')
('는/Josa', '매일/Noun')
('매일/Noun', '아침/Noun')
('아침/Noun', '지하철/Noun')
('지하철/Noun', '을/Josa')
('을/Josa', '탄다/Verb')
('탄다/Verb', '</s>')


##### 1) 데이터 불러오기
- 네이버 영화 리뷰 데이터셋 다운로드

In [32]:
""" 이건 Ubuntu나 Colab 환경인듯
%%time
!wget -nc -q https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
"""

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt") # !!

('ratings_train.txt', <http.client.HTTPMessage at 0x1cb50aaa940>)

##### 2) 텍스트만 추출

In [33]:
with codecs.open("ratings_train.txt", encoding='utf-8') as f: # codecs 패키지 : 대용량 파일을 조금씩 읽을 수 있게 해줌
  # (1) \n 기준 Lines Split + \t 기준 Line Split
  dataset = [line.split('\t') for line in f.read().splitlines()]
  # (2) Header 제외
  dataset = dataset[1:]

print("데이터셋 형식 : ", dataset[:10]) # [['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'], ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'], ['10265843', '너무재밓었다그래서보는것을추천한다', '0'], ['9045019', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0'], ['6483659', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', '1'], ['5403919', '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', '0'], ['7797314', '원작의 긴장감을 제대로 살려내지못했다.', '0'], ['9443947', '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네', '0'], ['7156791', '액션이 없는데도 재미 있는 몇안되는 영화', '1'], ['5912145', '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', '1']]

# (3) 텍스트만 추출
sentences = [data[1] for data in dataset]

print("텍스트 데이터 : ", sentences[:5]) # ['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '너무재밓었다그래서보는것을추천한다', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다']
print("문장 개수 : ", len(sentences)) # 150000

데이터셋 형식 :  [['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'], ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'], ['10265843', '너무재밓었다그래서보는것을추천한다', '0'], ['9045019', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0'], ['6483659', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', '1'], ['5403919', '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', '0'], ['7797314', '원작의 긴장감을 제대로 살려내지못했다.', '0'], ['9443947', '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네', '0'], ['7156791', '액션이 없는데도 재미 있는 몇안되는 영화', '1'], ['5912145', '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', '1']]
텍스트 데이터 :  ['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '너무재밓었다그래서보는것을추천한다', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다']
문장 개수 :  150000


##### 3) Tokenization + N-gram 변환

In [34]:
sentence_ngrams = []
for sentence in tqdm(sentences):
  sentence_okt = okt_tokenize(sentence)
  bigram = ngrams(sentence_okt, 2, pad_left=True, pad_right=True, left_pad_symbol="<s>", right_pad_symbol="</s>")
  sentence_ngrams += [b for b in bigram]

100%|██████████| 150000/150000 [04:18<00:00, 581.11it/s]


In [35]:
print(sentence_ngrams[:10])

[('<s>', '아/Exclamation'), ('아/Exclamation', '더빙/Noun'), ('더빙/Noun', '../Punctuation'), ('../Punctuation', '진짜/Noun'), ('진짜/Noun', '짜증나네요/Adjective'), ('짜증나네요/Adjective', '목소리/Noun'), ('목소리/Noun', '</s>'), ('<s>', '흠/Noun'), ('흠/Noun', '.../Punctuation'), ('.../Punctuation', '포스터/Noun')]


##### 4) 조건부 표본 개수

(1) 조건부 표본 개수

In [41]:
cfd_ngrams = ConditionalFreqDist(sentence_ngrams)

In [42]:
print(cfd_ngrams["<s>"].most_common(5))

[('정말/Noun', 2718), ('이/Noun', 2371), ('진짜/Noun', 2232), ('이/Determiner', 2115), ('영화/Noun', 2069)]


(2) 토큰 C 다음으로 가장 많이 등장하는 N개 단어 반환

In [43]:
def cfd_most_common(c, n, pos=None):
  if pos is None:
    # print(okt_tokenize(c))
    return cfd_ngrams[okt_tokenize(c)[0]].most_common(n) # [0]을 한 이유는 문장의 가장 처음 형태소만 따오기 위함
  else:
    return cfd_ngrams["/".join([c, pos])].most_common(n)

In [44]:
print(cfd_most_common("나", 10))

[('는/Josa', 831), ('의/Josa', 339), ('만/Josa', 213), ('에게/Josa', 148), ('에겐/Josa', 84), ('랑/Josa', 81), ('한테/Josa', 50), ('참/Verb', 45), ('이/Determiner', 44), ('와도/Josa', 43)]


##### 5) 조건부 확률 계산

(1) 조건부 확률 계산

In [45]:
cpd_ngrams = ConditionalProbDist(cfd_ngrams, MLEProbDist)

In [46]:
# - “.” 다음에 “</s>”가 올 확률 출력
print(cpd_ngrams[okt_tokenize(".")[0]].prob("</s>"))

0.39102658679807606


(2) 토큰 C 다음으로 토큰 W가 Bigram으로 함께 등장할 확률

In [47]:
def cpd_bigram_prob(context, next):
  context_okt_token = okt_tokenize(context)[0]
  next_okt_token = okt_tokenize(next)[0]
  return cpd_ngrams[context_okt_token].prob(next_okt_token)

In [48]:
print(cpd_bigram_prob("이", "영화"))

0.4010748656417948


In [49]:
print(cpd_bigram_prob("영화", "이"))

0.00015767585785521414


##### 6) 문장 생성기
- 조건부 확률 계산 -> 가장 확률이 높은 토큰열 추출 -> 문장 생성

In [50]:
def generate_sentence(seed=None, debug=False):
  # (1) 시드 초기화
  if seed is not None:
    import random
    random.seed(seed)
  # (2) 변수 초기화
  context = "<s>" # 시작 단어
  sentence = []
  # (3) 문장 생성
  while True:
    # 1] 시작 단어가 사전에 없는 경우 : 문장 생성 중단
    if context not in cpd_ngrams:
      break
    # 2] 다음 단어 생성
    next_word_pos = cpd_ngrams[context].generate()
    # 3] 다음 단어가 문장의 마지막인 경우 : 문장 생성 중단
    if next_word_pos == "</s>":
      break
    # 4] 다음 단어 분석
    next_word = next_word_pos.split("/")[0]
    next_pos = next_word_pos.split("/")[1]
    # 5] 단어 전처리 (조사, 어미 등은 제외하고, 각 토큰은 띄어쓰기로 구분해서 생성)
    if context == "<s>":
      sentence.append(next_word.title())
    elif context in ["`", "\"","'","("]:
      sentence.append(next_word)
    elif next_word in ["'", ".", ",", ")", ":", ";", "?"]:
      sentence.append(next_word)
    elif next_pos in ["Josa", "Punctuation", "Suffix"]:
      sentence.append(next_word)
    elif next_word_pos in ["임/Noun", "것/Noun", "는걸/Noun", "릴때/Noun", "되다/Verb", "이다/Verb", "하다/Verb", "이다/Adjective"]:
      sentence.append(next_word)
    else:
      sentence.append(" " + next_word)
    context = next_word_pos
    # 6] 결과 출력
    if debug:
      print(next_word_pos)

  return "".join(sentence)

In [51]:
print(generate_sentence(2))

도리까지 본 영화 너무... 뭔가.. 최고네요. 하지만.. 눈물 낫다는건 또 영화에 들지 않는다. 근데 뭐야 어떻게 그렇게 착했던 윤재랑은 에바 그린 드레스 소리 듣는거임""" 에리 욧의 미모로 합성 한 가수 노래와 흥행 놓친 영화다. 사투리 연기 하나 없는 ‘ 스피드 감 넘치는 스릴 넘치는 연기를 이해 되지 못 하시는 분보다 훨 재밌구만 평점을 망처 놓은 듯하다. 영화 보는이로 하여금 불편함을 느꼇을듯


In [52]:
generate_sentence(2, debug=True)

도리/Noun
까지/Josa
본/Verb
영화/Noun
너무/Adverb
.../Punctuation
뭔가/Noun
../Punctuation
최고/Noun
네/Suffix
요/Josa
./Punctuation
하지만/Conjunction
../Punctuation
눈물/Noun
낫다는건/Verb
또/Noun
영화/Noun
에/Josa
들지/Verb
않는다/Verb
./Punctuation
근데/Adverb
뭐/Noun
야/Josa
어떻게/Adjective
그렇게/Adverb
착했던/Adjective
윤재/Noun
랑은/Josa
에바/Noun
그린/Noun
드레스/Noun
소리/Noun
듣는거/Verb
임/Noun
"""/Punctuation
에리/Noun
욧의/Noun
미모/Noun
로/Josa
합성/Noun
한/Determiner
가수/Noun
노래/Noun
와/Josa
흥행/Noun
놓친/Verb
영화/Noun
다/Josa
./Punctuation
사투리/Noun
연기/Noun
하나/Noun
없는/Adjective
‘/Foreign
스피드/Noun
감/Noun
넘치는/Adjective
스릴/Noun
넘치는/Adjective
연기/Noun
를/Josa
이해/Noun
되지/Verb
못/VerbPrefix
하시는/Verb
분/Noun
보다/Josa
훨/Noun
재밌구만/Adjective
평점/Noun
을/Josa
망처/Noun
놓은/Verb
듯/Noun
하다/Verb
./Punctuation
영화/Noun
보는이로/Verb
하여금/Adverb
불편함을/Adjective
느꼇을듯/Noun


'도리까지 본 영화 너무... 뭔가.. 최고네요. 하지만.. 눈물 낫다는건 또 영화에 들지 않는다. 근데 뭐야 어떻게 그렇게 착했던 윤재랑은 에바 그린 드레스 소리 듣는거임""" 에리 욧의 미모로 합성 한 가수 노래와 흥행 놓친 영화다. 사투리 연기 하나 없는 ‘ 스피드 감 넘치는 스릴 넘치는 연기를 이해 되지 못 하시는 분보다 훨 재밌구만 평점을 망처 놓은 듯하다. 영화 보는이로 하여금 불편함을 느꼇을듯'

### 5. 결과
： 위 예제는 코퍼스 내의 등장 빈도에 기반하여 문장을 생성한다. bigram 언어 모델로 생성한 것이기 때문에 인접한 두 단어는 그나마 자연스럽지만 멀리 떨어진 단어와는 전혀 무관한 모습을 보인다. 또한 생성된 문장의 전체적인 문맥이 부자연스러우며 통사적으로 부적절한 모습도 보인다. 이는 코퍼스 내의 정보만으로 제한된 단어 조합만을 고려하는 N-gram 언어 모델의 한계로 보인다. 위 예제는 단순화를 위해 전처리와 규칙 처리를 최소화하였는데, 데이터셋을 늘리고 한국어 특징에 맞게 전처리를 진행한다면 보다 향상된 결과를 얻을 수 있을 것이다.


<참고>

> https://datascienceschool.net/view-notebook/a0c848e1e2d343d685e6077c35c4203b/


